In [6]:
%load_ext sql

import pandas as pd
import numpy as np

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [4]:
#example usage

%sql sqlite:///path/to/your/database.db

result = %sql SELECT * FROM employees WHERE salary > 50000;
dataframe = result.DataFrame()